In [30]:
from Bio import Entrez
import csv
import os
import shutil
import copy
import tarfile
import fnmatch

Search directory for files where the name (pmc id) matches our list

In [ ]:
# only search for pmcids in the file names in a directory
def extract_txt_files_using_only_pmcids_from_dir(input_list_pmcids, input_dir, output_dir):
    
    # create output dir if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # list of pmcids that have already been found
    previosly_found_pmcids = []

    for subdir, _, files in os.walk(input_dir):  
        for file in files:
            if file.endswith('.txt'):
                file_path = os.path.join(subdir, file)
                file_name = file.split('.')[0]

                # check if file is in input list of pmcids
                if file_name in input_list_pmcids and file_name not in previosly_found_pmcids:
                    shutil.copy(file_path, os.path.join(output_dir, file))
                    print(f'Copied {file_name} to {output_dir}')
                    previosly_found_pmcids.append(file_name)
    
    return previosly_found_pmcids

In [41]:
# test extract_txt_files_using_only_pmcids_from_dir
pmc_ids_Till = ["PMC7569388", "PMC7456324", "PMC7484793", "PMC9558369", "PMC7460440", "PMC7781625", "PMC7738495", "PMC7817203", "PMC7378201", "PMC6800427", "PMC7456744", "PMC6444128", "PMC6344609", "PMC6795496", "PMC6797844", "PMC8008609", "PMC7946422", "PMC8586732", "PMC7868734", "PMC7393571", "PMC7677513", "PMC6965490", "PMC7730263", "PMC7546531", "PMC6538646", "PMC6692052", ]
pmc_ids_Zeynep = ["PMC8989999", "PMC7612546", "PMC8103097", "PMC8128913", "PMC8514721", "PMC8678303", "PMC8699173", "PMC8708753", "PMC8777002", "PMC8778376", "PMC8855857", "PMC8915975", "PMC8941000", "PMC9031264", "PMC9081902", "PMC9100044", "PMC9103485", "PMC9105816", "PMC9213371", "PMC9259489", "PMC9259500", "PMC9319092", "PMC9320355","PMC9332067", "PMC9383973", "PMC9391071", "PMC9484756", "PMC9504724", "PMC9568547", "PMC9632171", "PMC9723555"]
pmc_ids_Sabrina = ["PMC9921660", "PMC9900325" ,"PMC10025556", "PMC9934417", "PMC10096730", "PMC9966613", "PMC10063449", "PMC10110899", "PMC10064692", "PMC9992512", "PMC9207268", "PMC9672454", "PMC9361767", "PMC8586732", "PMC8544895", "PMC8157445", "PMC8621464", "PMC9011687", "PMC7612573", "PMC8313584", "PMC7889221", "PMC8588834", "PMC8145447", "PMC8656578", "PMC10077291", "PMC8459765", "PMC7611097", "PMC9017205", "PMC8533893", "PMC8651779", "PMC8313253", "PMC8401523", "PMC7612847", "PMC8131268"]

input_list_pmcids = pmc_ids_Till + pmc_ids_Zeynep + pmc_ids_Sabrina
input_dir = '/Users/tillohlendorf/Downloads/PMC009xxxxxx'
output_dir = '/Users/tillohlendorf/Downloads/PMC009xxxxxx_filtered_from_dir'

previosly_found_pmcids = extract_txt_files_using_only_pmcids_from_dir(input_list_pmcids, input_dir, output_dir)

Copied PMC9207268 to /Users/tillohlendorf/Downloads/PMC009xxxxxx_filtered_from_dir
Copied PMC9568547 to /Users/tillohlendorf/Downloads/PMC009xxxxxx_filtered_from_dir
Copied PMC9992512 to /Users/tillohlendorf/Downloads/PMC009xxxxxx_filtered_from_dir
Copied PMC9259489 to /Users/tillohlendorf/Downloads/PMC009xxxxxx_filtered_from_dir
Copied PMC9017205 to /Users/tillohlendorf/Downloads/PMC009xxxxxx_filtered_from_dir
Copied PMC9319092 to /Users/tillohlendorf/Downloads/PMC009xxxxxx_filtered_from_dir
Copied PMC9259500 to /Users/tillohlendorf/Downloads/PMC009xxxxxx_filtered_from_dir
Copied PMC9332067 to /Users/tillohlendorf/Downloads/PMC009xxxxxx_filtered_from_dir
Copied PMC9100044 to /Users/tillohlendorf/Downloads/PMC009xxxxxx_filtered_from_dir
Copied PMC9031264 to /Users/tillohlendorf/Downloads/PMC009xxxxxx_filtered_from_dir
Copied PMC9934417 to /Users/tillohlendorf/Downloads/PMC009xxxxxx_filtered_from_dir
Copied PMC9361767 to /Users/tillohlendorf/Downloads/PMC009xxxxxx_filtered_from_dir
Copi

Searches directory for matching files names (pmc id) and for pm_id match in text file
Problem: the code creates many FPs, but I dont know how to stop that, because the txt files do not have a uniform way of mentioning the pm id 

In [8]:
# use multiple encodings to read files
def extract_txt_files(input_list_pmids,input_list_pmcids, input_dir, output_dir):
    
    # create output dir if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # list of pmcids that have already been found
    previosly_found_pmcids = []

    encodings = ['utf-8', 'latin-1']  # list of encodings to try

    for subdir, _, files in os.walk(input_dir):  
        for file in files:
            if file.endswith('.txt'):
                file_path = os.path.join(subdir, file)
                file_name = file.split('.')[0]

                # check if file is in input list of pmcids
                if file_name in input_list_pmcids and file_name not in previosly_found_pmcids:
                    shutil.copy(file_path, os.path.join(output_dir, file))
                    print(f'Copied {file_name} to {output_dir}')
                    previosly_found_pmcids.append(file_name)

                # check if file contains pmids in input list of pmids
                content = None
                for encoding in encodings:
                    try:
                        with open(file_path, 'r', encoding=encoding) as f:
                            content = f.read()
                        break  # if the file was successfully read, break the loop
                    except UnicodeDecodeError:
                        continue  # if an error occurred, try the next encoding

                if content is None:  # if none of the encodings worked, skip the file
                    print(f'Skipped {file_name} due to encoding errors')
                    continue

                if (any(pm_id in content for pm_id in input_list_pmids) and file_name not in previosly_found_pmcids):
                    
                    # add txt files with pmids to output dir
                    shutil.copy(file_path, os.path.join(output_dir, file))
                    print(f'Copied {file_name} to {output_dir}')
                    
                    # keep track of added files
                    previosly_found_pmcids.append(file_name)
    
    return previosly_found_pmcids

# Problem: many FPs are generated by searching for pmids in the text 

In [ ]:
# test extract_txt_files
pmc_ids_Till = ["PMC7569388", "PMC7456324", "PMC7484793", "PMC9558369", "PMC7460440", "PMC7781625", "PMC7738495", "PMC7817203", "PMC7378201", "PMC6800427", "PMC7456744", "PMC6444128", "PMC6344609", "PMC6795496", "PMC6797844", "PMC8008609", "PMC7946422", "PMC8586732", "PMC7868734", "PMC7393571", "PMC7677513", "PMC6965490", "PMC7730263", "PMC7546531", "PMC6538646", "PMC6692052", ]
pmc_ids_Zeynep = ["PMC8989999", "PMC7612546", "PMC8103097", "PMC8128913", "PMC8514721", "PMC8678303", "PMC8699173", "PMC8708753", "PMC8777002", "PMC8778376", "PMC8855857", "PMC8915975", "PMC8941000", "PMC9031264", "PMC9081902", "PMC9100044", "PMC9103485", "PMC9105816", "PMC9213371", "PMC9259489", "PMC9259500", "PMC9319092", "PMC9320355","PMC9332067", "PMC9383973", "PMC9391071", "PMC9484756", "PMC9504724", "PMC9568547", "PMC9632171", "PMC9723555"]
pmc_ids_Sabrina = ["PMC9921660", "PMC9900325" ,"PMC10025556", "PMC9934417", "PMC10096730", "PMC9966613", "PMC10063449", "PMC10110899", "PMC10064692", "PMC9992512", "PMC9207268", "PMC9672454", "PMC9361767", "PMC8586732", "PMC8544895", "PMC8157445", "PMC8621464", "PMC9011687", "PMC7612573", "PMC8313584", "PMC7889221", "PMC8588834", "PMC8145447", "PMC8656578", "PMC10077291", "PMC8459765", "PMC7611097", "PMC9017205", "PMC8533893", "PMC8651779", "PMC8313253", "PMC8401523", "PMC7612847", "PMC8131268"]

input_list_pmcids = pmc_ids_Till + pmc_ids_Zeynep + pmc_ids_Sabrina
input_list_pmids = ['32111634', '32615090', '32859898', '32913197', '33259205', '32619440', '32823573', '32205883', '33382950', '33319778', '33443161', '32703946', '31628411', '31857433', '30852164', '31519811', '30673913', '31578309', '31681563', '34946037', '37938227', '34668583', '34944797', '34658896', '34739342', '33949105', '34001868', '34739338', '34857933', '33785070', '33646117', '34694375', '33075129', '32776003', '32376096', '33240188', '31998276', '33260635', '32376073', '34694375', '36332011', '33627512', '34039428', '34836426', '34806775', '34580292', '34580292', '34000281', '34282029', '34312385', '33550886', '34777363', '33925452', '34885050', '37073345', '34339295', '34111652', '34021796', '34099917', '34847376', '35475000', '34105518', '33137688', '34016512', '32622559', '34680258', '34616012', '34081399', '34158656', '33662870', '34442780', '34646011', '34093985', '34043764', '34467251', '34767456', '33469015', '34115827', '33733474', '34946037', '37938227', '34668583', '34944797', '34658896', '34739342', '33949105', '34001868', '34739338', '34857933', '33785070', '33646117', '34694375', '33075129', '32776003', '32376096', '33240188', '31998276', '33260635', '32376073', '36771343', '36622087', '36950387', '36795699', '37049583', '36851429', '36914891', '37918887', '37081886', '37004103', '36797499', '35734174', '35015907', '35939623', '35393532', '35840661', '35459547', '35893909', '34927460', '35917163', '35086950', '35122074', '34774804', '36223724', '35572572', '35565377', '35888707', '36081215', '36241650', '36067550', '35672519', '35288718', '36587707', '35990627']
input_dir = '/Users/tillohlendorf/Downloads/PMC009xxxxxx'
output_dir = '/Users/tillohlendorf/Downloads/PMC009xxxxxx_filtered'

previosly_found_pmcids = extract_txt_files(input_list_pmids,input_list_pmcids, input_dir, output_dir)

This function also only searched for matching file names (pmc ids) but it searches in the tar file. I cant get it to work however.

In [37]:
# only search for pmcids in the file names in a tar.gz file
def extract_txt_files_using_only_pmcids_from_tar(input_list_pmcids, input_tar_gz, output_dir):
    
    # create output dir if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # list of pmcids that have already been found
    previosly_found_pmcids = []

    with tarfile.open(input_tar_gz, 'r:gz') as tar:
        for member in tar.getmembers():
            if member.isfile() and member.name.endswith('.txt'):
                file_name = os.path.basename(member.name).split('.')[0]
                print(file_name)

                # check if file is in input list of pmcids
                if file_name in input_list_pmcids and file_name not in previosly_found_pmcids:
                    tar.extract(member, path=output_dir)
                    print(f'Copied {file_name} to {output_dir}')
                    previosly_found_pmcids.append(file_name)
    
    return previosly_found_pmcids

In [38]:
# test extract_txt_files_using_only_pmcids_from_tar
pmc_ids_Till = ["PMC7569388", "PMC7456324", "PMC7484793", "PMC9558369", "PMC7460440", "PMC7781625", "PMC7738495", "PMC7817203", "PMC7378201", "PMC6800427", "PMC7456744", "PMC6444128", "PMC6344609", "PMC6795496", "PMC6797844", "PMC8008609", "PMC7946422", "PMC8586732", "PMC7868734", "PMC7393571", "PMC7677513", "PMC6965490", "PMC7730263", "PMC7546531", "PMC6538646", "PMC6692052", ]
pmc_ids_Zeynep = ["PMC8989999", "PMC7612546", "PMC8103097", "PMC8128913", "PMC8514721", "PMC8678303", "PMC8699173", "PMC8708753", "PMC8777002", "PMC8778376", "PMC8855857", "PMC8915975", "PMC8941000", "PMC9031264", "PMC9081902", "PMC9100044", "PMC9103485", "PMC9105816", "PMC9213371", "PMC9259489", "PMC9259500", "PMC9319092", "PMC9320355","PMC9332067", "PMC9383973", "PMC9391071", "PMC9484756", "PMC9504724", "PMC9568547", "PMC9632171", "PMC9723555"]
pmc_ids_Sabrina = ["PMC9921660", "PMC9900325" ,"PMC10025556", "PMC9934417", "PMC10096730", "PMC9966613", "PMC10063449", "PMC10110899", "PMC10064692", "PMC9992512", "PMC9207268", "PMC9672454", "PMC9361767", "PMC8586732", "PMC8544895", "PMC8157445", "PMC8621464", "PMC9011687", "PMC7612573", "PMC8313584", "PMC7889221", "PMC8588834", "PMC8145447", "PMC8656578", "PMC10077291", "PMC8459765", "PMC7611097", "PMC9017205", "PMC8533893", "PMC8651779", "PMC8313253", "PMC8401523", "PMC7612847", "PMC8131268"]

input_list_pmcids = pmc_ids_Till + pmc_ids_Zeynep + pmc_ids_Sabrina
input_tar_gz = '/Users/tillohlendorf/Downloads/oa_comm_txt.incr.2023-12-18.tar.gz'
output_dir = '/Users/tillohlendorf/Downloads/PMC009xxxxxx_filtered_from_tar'

previosly_found_pmcids = extract_txt_files_using_only_pmcids_from_tar(input_list_pmcids, input_tar_gz, output_dir)

PMC9841771
PMC9994490
PMC9994491
PMC9994492
PMC9994493
PMC9994494
PMC9994495
PMC9994496
PMC9994497
PMC9994498
PMC9994499
PMC9994500
PMC9994501
PMC9994502
PMC9994503
PMC9994504
PMC9994505
PMC9994506
PMC9994507
PMC9994509
PMC9994510
PMC9994511
PMC9994512
PMC9994513
PMC9994514
PMC9994515
PMC9994516
PMC9994517
PMC9994518
PMC9994519
PMC9994520
PMC9994521
PMC9994522
PMC9994523
PMC9994524
PMC9994525
PMC9994526
PMC9994527
PMC9998048
PMC9998049
PMC9998050
PMC9998051
PMC9998052
PMC9998053
PMC9998054
PMC9998055
PMC9998056
PMC9998057
PMC9998058
PMC9998059
PMC9998060
PMC9998061
PMC9998062
PMC9998063
PMC9998064
PMC9998065
PMC9998067
PMC9998068
PMC10043885
PMC10043886
PMC10043887
PMC10043888
PMC10043889
PMC10043890
PMC10043891
PMC10043892
PMC10043893
PMC10043894
PMC10043895
PMC10043896
PMC10043897
PMC10043898
PMC10043899
PMC10043900
PMC10043901
PMC10043902
PMC10043903
PMC10043905
PMC10043906
PMC10043907
PMC10043908
PMC10043909
PMC10043910
PMC10043911
PMC10043912
PMC10043913
PMC10043914
PMC10043915
PM